# Arithmetic coding
In this file, we will implement the vanilla Arithmetic coding and its variants, such as:
- Finite-accuracy  arithmetic coding
- Adaptive arithmetic coder

## 1. Vanilla arithmetic
### 1.1. Encoder

In [23]:
def get_n_first_binary(x, n):
    """Convert number from decimal to binary

    Args:
        x (float): number from 0 to 1
        n (_type_): number of bits

    Returns:
        result: _description_
    """
    result = []
    cmp = 0.5
    for i in range(n):
        if x >= cmp:
            result.append("1")
            x -= cmp
        else:
            result.append("0")
        cmp = cmp/2
    return result

In [34]:
import math
def arith_encoder(x, p0, verbose=False):
    """_summary_
    arithmetic encoder
    Args:
        x (array): message
        p0 (float): probability of 0 in the binary message
        verbose (binary): print some description
    Return:
        encoded message
    """
    if verbose:
        print("-----------Start encoding----------")
    l, h = 0, 1
    for i in range(len(x)):
        if x[i] == '0':
            h = l + p0*(h-l)
            if verbose:
                print(f"Index {i}: update h = l + p0*(h-l) = {h}")
        else:
            l = l + p0*(h-l)
            if verbose:
                print(f"Index {i}: update l = l + p0*(h-l) = {l}")
    mn = (h+l)/2
    if verbose:
        print(f"Done looping. Calculate the mid point mn = {mn}")
    lamda = math.ceil(-math.log2(h-l)) + 1
    if verbose:
        print(f"Number of bit to truncate: {lamda}")
    code = get_n_first_binary(mn, lamda)
    if verbose:
        print("Done!!!")
    return code
    

## 1.2. Decoder

In [35]:
def arith_decoder(N, p0, C_message, verbose=False):
    """Arithmetic decoder

    Args:
        N (int): length of original message
        p0 (float): [0, 1] probability of 0 in the original message
        C_message (array): encoded message
        verbose (binary): print some description
    """
    if verbose:
        print("-----------Start decoding----------")
    c = 0
    for i in range(len(C_message)):
        if C_message[i] == "1":
            c += math.pow(2, -i-1)
    if verbose:
        print(f"c = (0,C_message) = {c}")
        print("Initialize variable x, l = 0, h = 1")
    x = []
    l, h = 0, 1
    for i in range(N):
        if c >= l and c <= l + p0*(h-l):
            x.append("0")
            h = l + p0*(h-l)
            if verbose:
                print(f"Step {i} c in range [l, l+p0*(h-l)], append 0, update h")
        else:
            x.append("1")
            l = l + p0*(h-l)
            if verbose:
                print(f"Step {i} c in range [l+p0*(h-l), h], append 1, update l")
    if verbose:
        print("Done!!!")
    return x

In [36]:
message = "1001010110100"
p0 = 0.4
print(f"Original message: {message}")
encoded_message = arith_encoder(message, p0, True)
print(f"Encoded message: {encoded_message}")

decoded_message = arith_decoder(len(message), p0, encoded_message, True)
print(f"Decoded message: {decoded_message}")

Original message: 1001010110100
-----------Start encoding----------
Index 0: update l = l + p0*(h-l) = 0.4
Index 1: update h = l + p0*(h-l) = 0.64
Index 2: update h = l + p0*(h-l) = 0.496
Index 3: update l = l + p0*(h-l) = 0.4384
Index 4: update h = l + p0*(h-l) = 0.46144
Index 5: update l = l + p0*(h-l) = 0.447616
Index 6: update h = l + p0*(h-l) = 0.45314560000000004
Index 7: update l = l + p0*(h-l) = 0.44982784000000003
Index 8: update l = l + p0*(h-l) = 0.45115494400000006
Index 9: update h = l + p0*(h-l) = 0.4519512064
Index 10: update l = l + p0*(h-l) = 0.45147344896
Index 11: update h = l + p0*(h-l) = 0.451664551936
Index 12: update h = l + p0*(h-l) = 0.45154989015040004
Done looping. Calculate the mid point mn = 0.45151166955520006
Number of bit to truncate: 15
Done!!!
Encoded message: ['0', '1', '1', '1', '0', '0', '1', '1', '1', '0', '0', '1', '0', '1', '1']
-----------Start decoding----------
c = (0,C_message) = 0.451507568359375
Initialize variable x, l = 0, h = 1
Step 0 c 